In [1]:
# Import libraries and Data
!pip install tensorflow
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [3]:
# path for data folder
path= r"C:\Users\Holly\OneDrive\David's work\project 1 machine learning\02 Data"

In [5]:
# Importing unscaled weather features data 
unscaled= pd.read_csv(os.path.join(path,r"C:\Users\Holly\OneDrive\David's work\project 1 machine learning\02 Data\Dataset-weather-prediction-dataset-processed.csv"))

In [7]:
#Importing pleasant weather 'answer' dataset 
prediction= pd.read_csv(os.path.join(path,r"C:\Users\Holly\OneDrive\David's work\project 1 machine learning\02 Data\Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv"))

In [9]:
unscaled.shape

(22950, 170)

In [11]:
unscaled.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.018,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.018,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.018,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


## Data Wrangle

In [13]:
# Remove weather stations not included in "pleasant weather" answers
unscaled = unscaled.drop(['GDANSK_cloud_cover', 'GDANSK_humidity', 'GDANSK_precipitation', 'GDANSK_snow_depth', 'GDANSK_temp_mean', 'GDANSK_temp_min', 'GDANSK_temp_max',
                        'ROMA_cloud_cover', 'ROMA_wind_speed', 'ROMA_humidity', 'ROMA_pressure', 'ROMA_sunshine', 'ROMA_temp_mean',
                        'TOURS_wind_speed', 'TOURS_humidity', 'TOURS_pressure', 'TOURS_global_radiation', 'TOURS_precipitation', 'TOURS_temp_mean', 'TOURS_temp_min', 'TOURS_temp_max'], axis=1)

In [15]:
# Check the pleasant dataset
prediction.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
prediction.shape

(22950, 16)

In [19]:
unscaled.isnull().sum()

DATE                   0
MONTH                  0
BASEL_cloud_cover      0
BASEL_wind_speed       0
BASEL_humidity         0
                      ..
VALENTIA_snow_depth    0
VALENTIA_sunshine      0
VALENTIA_temp_mean     0
VALENTIA_temp_min      0
VALENTIA_temp_max      0
Length: 149, dtype: int64

In [21]:
prediction.isnull().sum()

DATE                           0
BASEL_pleasant_weather         0
BELGRADE_pleasant_weather      0
BUDAPEST_pleasant_weather      0
DEBILT_pleasant_weather        0
DUSSELDORF_pleasant_weather    0
HEATHROW_pleasant_weather      0
KASSEL_pleasant_weather        0
LJUBLJANA_pleasant_weather     0
MAASTRICHT_pleasant_weather    0
MADRID_pleasant_weather        0
MUNCHENB_pleasant_weather      0
OSLO_pleasant_weather          0
SONNBLICK_pleasant_weather     0
STOCKHOLM_pleasant_weather     0
VALENTIA_pleasant_weather      0
dtype: int64

In [23]:
# Extract the different observation types

observation_types = ['cloud_cover', 'wind_speed', 'humidity', 'pressure',
                     'global_radiation', 'precipitation', 'snow_depth', 
                     'sunshine', 'temp_mean', 'temp_min', 'temp_max']

In [25]:
# Create a dictionary to store the count of stations for each observation type
station_counts = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in unscaled.columns if col.endswith(obs)]
    
    # Count the number of stations (i.e., the number of columns) for the current observation type
    station_counts[obs] = len(columns)

# Print the count of stations for each observation type
print("Number of stations covered by each observation type:")
for obs, count in station_counts.items():
    print(f"{obs}: {count} stations")


Number of stations covered by each observation type:
cloud_cover: 14 stations
wind_speed: 9 stations
humidity: 14 stations
pressure: 14 stations
global_radiation: 15 stations
precipitation: 15 stations
snow_depth: 6 stations
sunshine: 15 stations
temp_mean: 15 stations
temp_min: 15 stations
temp_max: 15 stations


In [27]:
# Get a list of columns containing 'wind_speed' or 'snow_depth'
cols_to_drop = [col for col in unscaled.columns if '_wind_speed' in col or '_snow_depth' in col]

# Drop the columns
unscaled = unscaled.drop(cols_to_drop, axis=1)

In [29]:
unscaled.shape

(22950, 134)

In [31]:
# Find the stations with the above entries missing
# Get all column names
all_columns = unscaled.columns.tolist()
# Exclude 'DATE' and 'MONTH' columns
all_columns = [col for col in all_columns if col not in ['DATE', 'MONTH']]  
# Extract unique weather station names
weather_stations = set()  # Use a set to automatically store only unique values
for col in all_columns:
    station_name = col.split('_')[0]  # Split the column name at the underscore and take the first part
    weather_stations.add(station_name)

# Print the list of weather stations
print(weather_stations)

{'BELGRADE', 'BUDAPEST', 'OSLO', 'VALENTIA', 'DUSSELDORF', 'MAASTRICHT', 'STOCKHOLM', 'KASSEL', 'MADRID', 'BASEL', 'MUNCHENB', 'LJUBLJANA', 'SONNBLICK', 'DEBILT', 'HEATHROW'}


In [33]:
# Find stations missing observation types
observation_types = ['cloud_cover', 'humidity', 'pressure']

missing_stations_by_observation = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in unscaled.columns if col.endswith(obs)]
    
    # Extract station names by removing the observation type from the column names
    station_names = set([col.replace(f'_{obs}', '') for col in columns])
    
    # Identify stations that are in all_stations but missing from the current observation type
    missing_stations = weather_stations - station_names
    
    # Store the missing station names in the dictionary
    missing_stations_by_observation[obs] = missing_stations

# Print the missing station names for each observation type
for obs, missing_stations in missing_stations_by_observation.items():
    print(f"\nStations missing from {obs}:")
    if missing_stations:
        for station in missing_stations:
            print(station)
    else:
        print("None")


Stations missing from cloud_cover:
KASSEL

Stations missing from humidity:
STOCKHOLM

Stations missing from pressure:
MUNCHENB


In [35]:
unscaled.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,19600102,1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,19600103,1,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,19600104,1,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,19600105,1,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [37]:
# Cloud cover is the start of a stations data, Kassel is next to Heathrow, find the position of Heathrow_temp_max for the insertion of Kassel_cloud_cover
unscaled.columns.get_loc('HEATHROW_temp_max')

55

In [39]:
# Find the position for insertion of Stockholm humidity
unscaled.columns.get_loc('STOCKHOLM_cloud_cover') #humidity is 1 after cloud cover so (result +1)

117

In [41]:
# Find position for Munchenb pressure
unscaled.columns.get_loc('MUNCHENB_cloud_cover') # pressure is 2 after cloud cover so (result +2)

91

In [43]:
# Insert new columns into "unscaled" at specific positions.
# The data for these new columns is taken from weather stations they are close to

unscaled.insert(56,'KASSEL_cloud_cover', unscaled['DUSSELDORF_cloud_cover'])
unscaled.insert(119, 'STOCKHOLM_humidity', unscaled['OSLO_humidity'])
unscaled.insert(94,'MUNCHENB_pressure',unscaled['BASEL_pressure'])

In [45]:
unscaled.columns.tolist()

['DATE',
 'MONTH',
 'BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSS

In [47]:
unscaled.shape

(22950, 137)

In [49]:
prediction.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
# drop unneeded column from second dataset
prediction.drop(columns = 'DATE', inplace = True)

In [53]:
# Now, confirm drop
prediction.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
prediction.shape

(22950, 15)

In [57]:
# Export cleaned dataset
unscaled.to_csv(os.path.join(path, 'weather_cleaned.csv'), index=False)

## Data Reshape

In [71]:
# Creat an 'X' matrix by reloading and naming our data 'X'
X = pd.read_csv(os.path.join(path,'weather_cleaned.csv'), index_col=False)

In [73]:
X.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [75]:
y = prediction

In [77]:
X.shape

(22950, 135)

In [79]:
# Turn X and y into arrays
X = np.array(X)
y = np.array(y)
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [81]:
X = X.reshape(-1,15,9)

In [83]:
# Verify Shape
X.shape

(22950, 15, 9)

In [85]:
# Verify Shape
y.shape

(22950, 15)

In [87]:
X

array([[[  7.    ,   0.85  ,   1.018 , ...,   6.5   ,   0.8   ,
          10.9   ],
        [  1.    ,   0.81  ,   1.0195, ...,   3.7   ,  -0.9   ,
           7.9   ],
        [  4.    ,   0.67  ,   1.017 , ...,   2.4   ,  -0.4   ,
           5.1   ],
        ...,
        [  4.    ,   0.73  ,   1.0304, ...,  -5.9   ,  -8.5   ,
          -3.2   ],
        [  5.    ,   0.98  ,   1.0114, ...,   4.2   ,   2.2   ,
           4.9   ],
        [  5.    ,   0.88  ,   1.0003, ...,   8.5   ,   6.    ,
          10.9   ]],

       [[  6.    ,   0.84  ,   1.018 , ...,   6.1   ,   3.3   ,
          10.1   ],
        [  6.    ,   0.84  ,   1.0172, ...,   2.9   ,   2.2   ,
           4.4   ],
        [  4.    ,   0.67  ,   1.017 , ...,   2.3   ,   1.4   ,
           3.1   ],
        ...,
        [  6.    ,   0.97  ,   1.0292, ...,  -9.5   , -10.5   ,
          -8.5   ],
        [  5.    ,   0.62  ,   1.0114, ...,   4.    ,   3.    ,
           5.    ],
        [  7.    ,   0.91  ,   1.0007, ...,   8.

## Data Splitting 

In [90]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [92]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


## Keras Model

In [95]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [97]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 14, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,831 (11.06 KB)

 Trainable params: 2,831 (11.06 KB)

 Non-trainable params: 0 (0.00 B)

## Compile and Running Model

In [100]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [102]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 12s - 11ms/step - accuracy: 0.1334 - loss: 5412.6689
Epoch 2/30
1076/1076 - 8s - 8ms/step - accuracy: 0.1339 - loss: 53711.4922
Epoch 3/30
1076/1076 - 12s - 11ms/step - accuracy: 0.1329 - loss: 184324.0781
Epoch 4/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1311 - loss: 394287.6562
Epoch 5/30
1076/1076 - 7s - 6ms/step - accuracy: 0.1303 - loss: 699204.8750
Epoch 6/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1340 - loss: 1131961.5000
Epoch 7/30
1076/1076 - 8s - 8ms/step - accuracy: 0.1318 - loss: 1659987.1250
Epoch 8/30
1076/1076 - 7s - 6ms/step - accuracy: 0.1300 - loss: 2337215.5000
Epoch 9/30
1076/1076 - 7s - 6ms/step - accuracy: 0.1287 - loss: 3101672.5000
Epoch 10/30
1076/1076 - 7s - 7ms/step - accuracy: 0.1301 - loss: 4050989.7500
Epoch 11/30
1076/1076 - 6s - 5ms/step - accuracy: 0.1322 - loss: 5097046.0000
Epoch 12/30
1076/1076 - 7s - 6ms/step - accuracy: 0.1305 - loss: 6289769.0000
Epoch 13/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1347 - loss: 7685916.5000

## Confusion Matrix

In [105]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [107]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [109]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Pred        DEBILT  DUSSELDORF  HEATHROW  KASSEL  LJUBLJANA  MAASTRICHT  \
True                                                                      
BASEL          255          11         3      59       1769         307   
BELGRADE        15           0         0       0        698          29   
BUDAPEST         3           0         0       0         94           8   
DEBILT           3           0         0       0         33           3   
DUSSELDORF       1           0         0       0          6           5   
HEATHROW         5           0         0       0          8           4   
KASSEL           1           0         0       0          6           1   
LJUBLJANA        1           0         0       0         23           2   
MAASTRICHT       0           0         0       0          5           0   
MADRID          30           0         0       1         72          23   
MUNCHENB         0           0         0       0          6

## Keras Model Retrials

In [112]:
epochs = 30
batch_size = 16
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # Options: sigmoid, tanh, softmax, relu

In [114]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 14, 16)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,851 (7.23 KB)

 Trainable params: 1,851 (7.23 KB)

 Non-trainable params: 0 (0.00 B)

In [116]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [118]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 12s - 11ms/step - accuracy: 0.1199 - loss: 310.2769
Epoch 2/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1312 - loss: 2824.5261
Epoch 3/30
1076/1076 - 8s - 7ms/step - accuracy: 0.1361 - loss: 9457.5664
Epoch 4/30
1076/1076 - 7s - 6ms/step - accuracy: 0.1382 - loss: 20126.2129
Epoch 5/30
1076/1076 - 8s - 7ms/step - accuracy: 0.1405 - loss: 37439.6953
Epoch 6/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1354 - loss: 58767.5352
Epoch 7/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1329 - loss: 85150.3203
Epoch 8/30
1076/1076 - 6s - 5ms/step - accuracy: 0.1390 - loss: 121737.5781
Epoch 9/30
1076/1076 - 6s - 5ms/step - accuracy: 0.1383 - loss: 164103.5625
Epoch 10/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1393 - loss: 212404.8750
Epoch 11/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1329 - loss: 271060.2500
Epoch 12/30
1076/1076 - 6s - 5ms/step - accuracy: 0.1368 - loss: 339110.1562
Epoch 13/30
1076/1076 - 6s - 6ms/step - accuracy: 0.1332 - loss: 412769.4375
Epoch 14/30
1076/

In [120]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [122]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL           5       142       611      89         218       226      41   
BELGRADE        0       139        12      38          21         0       5   
BUDAPEST        0        16         1       6          10         0       0   
DEBILT          0         4         0       7           4         0       0   
DUSSELDORF      0         1         0       3           3         0       0   
HEATHROW        0         1         0       9          12         0       1   
KASSEL          0         2         0       1           1         0       0   
LJUBLJANA       0         6         0       1           3         0       0   
MAASTRICHT      0         1         0       0           3         0       0   
MADRID          0         7         4       6          66        20       0   
MUNCHENB   

In [124]:
epochs = 30
batch_size = 16
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [126]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 14, 16)         │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,191 (24.18 KB)

 Trainable params: 6,191 (24.18 KB)

 Non-trainable params: 0 (0.00 B)

In [128]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [130]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 12s - 11ms/step - accuracy: 0.0613 - loss: 24.0554
Epoch 2/30
1076/1076 - 10s - 9ms/step - accuracy: 0.0049 - loss: 24.4627
Epoch 3/30
1076/1076 - 7s - 7ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 4/30
1076/1076 - 7s - 7ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 5/30
1076/1076 - 7s - 6ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 6/30
1076/1076 - 7s - 6ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 7/30
1076/1076 - 7s - 6ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 8/30
1076/1076 - 7s - 7ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 9/30
1076/1076 - 7s - 6ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 10/30
1076/1076 - 7s - 6ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 11/30
1076/1076 - 7s - 7ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 12/30
1076/1076 - 7s - 6ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 13/30
1076/1076 - 9s - 8ms/step - accuracy: 0.0016 - loss: 24.4309
Epoch 14/30
1076/1076 - 6s - 6ms/step - accuracy: 0.0016 

In [132]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [134]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Pred        BELGRADE  DUSSELDORF  HEATHROW  LJUBLJANA  MAASTRICHT  MADRID  \
True                                                                        
BASEL              4           1       111         80         860     109   
BELGRADE           9           0        31         68         636       0   
BUDAPEST           1           0         4          4          95       0   
DEBILT             0           0         0          2          47       0   
DUSSELDORF         0           0         0          1          11       0   
HEATHROW           0           0         0          0          20       0   
KASSEL             0           0         0          0           6       0   
LJUBLJANA          0           0         0          2           5       0   
MAASTRICHT         0           0         0          0           1       0   
MADRID             2           0         2          1          58       0   
MUNCHENB           0           0   

In [136]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [138]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [140]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [142]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 14s - 13ms/step - accuracy: 0.0282 - loss: 21.1009
Epoch 2/30
1076/1076 - 9s - 8ms/step - accuracy: 0.1758 - loss: 24.9006
Epoch 3/30
1076/1076 - 8s - 8ms/step - accuracy: 0.1946 - loss: 24.4666
Epoch 4/30
1076/1076 - 10s - 9ms/step - accuracy: 0.1946 - loss: 25.6212
Epoch 5/30
1076/1076 - 11s - 10ms/step - accuracy: 0.1946 - loss: 25.5893
Epoch 6/30
1076/1076 - 11s - 10ms/step - accuracy: 0.1946 - loss: 25.5500
Epoch 7/30
1076/1076 - 12s - 11ms/step - accuracy: 0.1946 - loss: 25.1220
Epoch 8/30
1076/1076 - 9s - 9ms/step - accuracy: 0.1946 - loss: 23.5778
Epoch 9/30
1076/1076 - 11s - 10ms/step - accuracy: 0.1946 - loss: 23.5741
Epoch 10/30
1076/1076 - 11s - 10ms/step - accuracy: 0.1946 - loss: 23.5740
Epoch 11/30
1076/1076 - 11s - 10ms/step - accuracy: 0.1946 - loss: 23.5741
Epoch 12/30
1076/1076 - 10s - 10ms/step - accuracy: 0.1946 - loss: 23.5741
Epoch 13/30
1076/1076 - 12s - 11ms/step - accuracy: 0.1946 - loss: 23.5722
Epoch 14/30
1076/1076 - 11s - 10ms/step -

In [144]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [146]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  MADRID  MUNCHENB  VALENTIA
True                                                                     
BASEL        1446      2165        41       6       6        14         4
BELGRADE      886       206         0       0       0         0         0
BUDAPEST      191        23         0       0       0         0         0
DEBILT         80         2         0       0       0         0         0
DUSSELDORF     26         3         0       0       0         0         0
HEATHROW       66        16         0       0       0         0         0
KASSEL         10         1         0       0       0         0         0
LJUBLJANA      33        28         0       0       0         0         0
MAASTRICHT      5         4         0       0       0         0         0
MADRID        162       296         0       0       0         0         0
MUNCHENB        4         4         0       0       0         0        

In [148]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) # Options: sigmoid, tanh, softmax, relu

In [150]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [152]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [154]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 14s - 13ms/step - accuracy: 0.6181 - loss: 7473.1250
Epoch 2/30
1076/1076 - 8s - 7ms/step - accuracy: 0.6436 - loss: 79615.6562
Epoch 3/30
1076/1076 - 9s - 9ms/step - accuracy: 0.6438 - loss: 276341.9688
Epoch 4/30
1076/1076 - 9s - 8ms/step - accuracy: 0.6439 - loss: 600801.6250
Epoch 5/30
1076/1076 - 11s - 10ms/step - accuracy: 0.6439 - loss: 1080887.8750
Epoch 6/30
1076/1076 - 11s - 10ms/step - accuracy: 0.6439 - loss: 1708916.5000
Epoch 7/30
1076/1076 - 10s - 9ms/step - accuracy: 0.6440 - loss: 2512762.2500
Epoch 8/30
1076/1076 - 10s - 9ms/step - accuracy: 0.6440 - loss: 3492421.0000
Epoch 9/30
1076/1076 - 10s - 9ms/step - accuracy: 0.6440 - loss: 4698262.5000
Epoch 10/30
1076/1076 - 9s - 9ms/step - accuracy: 0.6440 - loss: 6076169.0000
Epoch 11/30
1076/1076 - 10s - 9ms/step - accuracy: 0.6440 - loss: 7757760.0000
Epoch 12/30
1076/1076 - 12s - 11ms/step - accuracy: 0.6439 - loss: 9654584.0000
Epoch 13/30
1076/1076 - 8s - 8ms/step - accuracy: 0.6440 - loss: 118

In [156]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [158]:
# Evaluate

print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Pred        BASEL
True             
BASEL        3682
BELGRADE     1092
BUDAPEST      214
DEBILT         82
DUSSELDORF     29
HEATHROW       82
KASSEL         11
LJUBLJANA      61
MAASTRICHT      9
MADRID        458
MUNCHENB        8
OSLO            5
STOCKHOLM       4
VALENTIA        1


In [160]:
epochs = 15
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='relu')) # Options: sigmoid, tanh, softmax, relu

In [162]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_5 (Conv1D)               │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 14, 16)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,851 (7.23 KB)

 Trainable params: 1,851 (7.23 KB)

 Non-trainable params: 0 (0.00 B)

In [164]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [166]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
4303/4303 - 34s - 8ms/step - accuracy: 0.4771 - loss: nan
Epoch 2/15
4303/4303 - 29s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 3/15
4303/4303 - 29s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/15
4303/4303 - 27s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/15
4303/4303 - 27s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/15
4303/4303 - 26s - 6ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/15
4303/4303 - 31s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/15
4303/4303 - 32s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/15
4303/4303 - 35s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/15
4303/4303 - 39s - 9ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/15
4303/4303 - 33s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 12/15
4303/4303 - 31s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 13/15
4303/4303 - 32s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 14/15
4303/4303 - 35s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 15/15
4303/4303 - 33s -

In [168]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [170]:
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Pred        BASEL
True             
BASEL        3682
BELGRADE     1092
BUDAPEST      214
DEBILT         82
DUSSELDORF     29
HEATHROW       82
KASSEL         11
LJUBLJANA      61
MAASTRICHT      9
MADRID        458
MUNCHENB        8
OSLO            5
STOCKHOLM       4
VALENTIA        1
